### python -m pip install pythonnet

In [13]:
from pathlib import Path
import logging
import json

In [14]:
data_folder = Path('./test_data/')
material_data = data_folder / 'material_data.json'
column_data = data_folder / 'column_data.json'
column_cross_section_data = data_folder / 'column_cross_section_data.json'
analytical_levels = data_folder / 'analytical_levels.json'

beam_cross_section_data = data_folder / 'beam_cross_section_data.json'
beam_data = data_folder / 'beam_data.json'
floor_data = data_folder / 'floor_data.json'
floor_cross_section_data = data_folder / 'floor_cross_section_data.json'
wall_data = data_folder / 'wall_data.json'
wall_cross_section_data = data_folder / 'wall_cross_section_data.json'

# add logging

In [15]:
# Create a logger
logger = logging.getLogger('etabs_logger')
logger.setLevel(logging.INFO)

# Check if handlers are already added to avoid duplicate logs
if not logger.handlers:
    # Create a file handler
    log_file_handler = logging.FileHandler('log.txt')
    log_file_handler.setLevel(logging.INFO)

    # Create a formatter and set it for the handler
    formatter = logging.Formatter('%(asctime)s | %(funcName)s | %(levelname)s | %(lineno)d | %(message)s')
    log_file_handler.setFormatter(formatter)

    # Add the handler to the logger
    logger.addHandler(log_file_handler)


logger.info('starting the program')





#start ETABS application.
myETABSObject.ApplicationStart()

#close the program.
ret = myETABSObject.ApplicationExit(False)


In [16]:
import clr
clr.AddReference("System.Runtime.InteropServices")
from System.Runtime.InteropServices import Marshal

#set the following path to the installed ETABS program directory
clr.AddReference(R'C:\Program Files\Computers and Structures\ETABS 22\ETABSv1.dll')
import ETABSv1 as etabs

#create API helper object
helper = etabs.cHelper(etabs.Helper())

try:
    myETABSObject = etabs.cOAPI(helper.GetObject("CSI.ETABS.API.ETABSObject"))
except:
    error_msg = "No running instance of the program found or failed to attach."
    print(error_msg)
    logger.error(error_msg)

#create SapModel object
SapModel = etabs.cSapModel(myETABSObject.SapModel)

N_mm_C = etabs.eUnits.N_mm_C
kN_m_C = etabs.eUnits.kN_m_C

display(myETABSObject)
display(SapModel)



# read columns from etabs

In [17]:
with open (column_data, 'r') as file:
    columns = json.load(file)

def get_etabs_column_concrete_section(section_data):
    size = str(section_data[1]['name']).split('_')[2]
    if str(section_data[1]['material_grade']).upper().startswith('FC'):
        if size.upper().startswith('D'):
            size = size[1:]
        name = f"C {size}-{section_data[1]['rebar']}-{section_data[1]['material_grade']}"
    else:
        name = f"C {size}-{section_data[1]['material_grade']}"
    return name

revit_col_section_name = {str(k):get_etabs_column_concrete_section((k,v)) for k,v in columns}
revit_col_section_name


GetAllFrames = {
    'NumberNames': 0,
    'MyName': [],
    'PropName': [],
    'StoryName': [],
    'PointName1': [],
    'PointName2': [],
    'Point1X': [],
    'Point1Y': [],
    'Point1Z': [],
    'Point2X': [],
    'Point2Y': [],
    'Point2Z': [],
    'Angle': [],
    'Offset1X': [],
    'Offset2X': [],
    'Offset1Y': [],
    'Offset2Y': [],
    'Offset1Z': [],
    'Offset2Z': [],
    'CardinalPoint': [],
    'csys': "Global"
}

ret = SapModel.FrameObj.GetAllFrames(**GetAllFrames)
frame_names = list(ret[2])
property_names = list(ret[3])

etabs_frame_section_name = {str(k).split('_')[0]:v for k,v in zip(frame_names,property_names)}

column_changed = []
for k,v in revit_col_section_name.items():
    if k in etabs_frame_section_name:
        if v != etabs_frame_section_name[k]:
            column_changed.append(k)

column_changed


['6864165', '7081867', '7081894']

# read beams from etabs

In [18]:
with open (beam_data, 'r') as file:
    beams = json.load(file)

def get_etabs_beam_cross_section(section_data):
    grade:str = section_data[1]['material_grade']
    if grade.upper().startswith('FC'):
        b = round(float(section_data[1]['b']))
        h = round(float(section_data[1]['h']))
        size = f"{b}X{h}"
        name = f"B {size}-{grade}"
    else:
        name = f"B {section_data[1]['name'][4:]}-{grade}"
    return name

revit_beam_section_name = {str(k):get_etabs_beam_cross_section((k,v)) for k,v in beams}



beam_changed = []
for k,v in revit_beam_section_name.items():
    if k in etabs_frame_section_name:
        if v != etabs_frame_section_name[k]:
            beam_changed.append(k)

beam_changed

['6918417', '6918428']

# read walls from etabs

In [19]:
with open (wall_data, 'r') as file:
    walls = json.load(file)

def get_etabs_wall_cross_section(section_data):
    grade:str = section_data[1]['material_grade']
    if grade.upper().startswith('FC'):
        thk = round(float(section_data[1]['thk']))
        name = f"W {thk}-{grade}"
    else:
        name = f"W {section_data[1]['name'][8:]}-{grade}"
    return name

revit_wall_section_name = {str(k):get_etabs_wall_cross_section((k,v)) for k,v in walls}


GetAllAreas = {
    'NumberNames': 0,
    'MyName': [],
    'DesignOrientation': [],
    'NumberBoundaryPts': 0,
    'PointDelimiter': [],
    'PointNames': [],
    'PointX': [],
    'PointY': [],
    'PointZ': []
}

ret,_,area_names,_,_,_,_,_,_,_ = SapModel.AreaObj.GetAllAreas(**GetAllAreas)
area_names = list(area_names)

PropName = ''
ret = [SapModel.AreaObj.GetProperty(str(x), PropName) for x in area_names]
etabs_area_section_name = [x[1] for x in ret]
etabs_area_section_name = {str(k).split('_')[0]:v for k,v in zip(area_names,etabs_area_section_name)}
etabs_area_section_name

wall_changed = []
for k,v in revit_wall_section_name.items():
    if k in etabs_area_section_name:
        if v != etabs_area_section_name[k]:
            wall_changed.append(k)
wall_changed


[]

In [20]:
#refresh view
ret = SapModel.View.RefreshView(0, False)
ret

0

# finish

In [21]:
SapModel = None
myETABSObject = None

In [22]:
';'.join(column_changed) if len(column_changed) > 0 else None

'6864165;7081867;7081894'

In [23]:
';'.join(wall_changed) if len(wall_changed) > 0 else None

In [24]:
';'.join(beam_changed) if len(beam_changed) > 0 else None

'6918417;6918428'